## ELMO

In [67]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv1D, Conv2D, MaxPooling2D, GlobalMaxPooling1D, MaxPool1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM
from keras.utils import to_categorical
from keras.engine import Layer
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K


import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import spacy
# from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
X_train_Q1 = pd.read_excel('../data/interim/X_train_Q1_clean.xlsx')
X_valid_Q1 = pd.read_excel('../data/interim/X_valid_Q1_clean.xlsx')

y_train_Q1 = pd.read_excel('../data/interim/y_train_Q1.xlsx')
y_valid_Q1 = pd.read_excel('../data/interim/y_valid_Q1.xlsx')

In [3]:
df = pd.concat([X_train_Q1, y_train_Q1.iloc[:,0:12]], axis = 1)

data_df = df

import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in data_df['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
data_df['preprocessed_comments']=preprocessed_synopsis

X_train = data_df[['preprocessed_comments']]
y_train = data_df.drop(['Comment', 'preprocessed_comments'], axis=1)


df_valid = pd.concat([X_valid_Q1, y_valid_Q1.iloc[:,:12]], axis = 1)

# pre-processing test data
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df_valid['Comment'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())
    
    
df_valid['preprocessed_comments']=preprocessed_synopsis

X_valid = df_valid[['preprocessed_comments']]
y_valid = df_valid.drop(columns=['Comment', 'preprocessed_comments'])

## Pre-Trained Model

In [4]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [5]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [6]:
# embeddings = elmo(
#     ["the cat is on the mat", "dogs are in the fog"],
#     signature="default",
#     as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [14]:
embeddings = elmo(np.asarray(X_train['preprocessed_comments']), signature='default', as_dict=True)['elmo']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [15]:
embeddings

<tf.Tensor 'module_apply_default_5/aggregation/mul_3:0' shape=(10376, 90, 1024) dtype=float32>

In [93]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [94]:
# Function to build model
def build_model(): 
    input_text = Input(shape=(1,))
    embedding = ElmoEmbeddingLayer()(input_text)
    dense = layers.Dense(256, activation='relu')(embedding)
    pred = layers.Dense(12, activation='sigmoid')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [95]:
# Build and fit
model = build_model()

AttributeError: module 'keras.backend' has no attribute 'tf'

## Simple Model

In [20]:
max_features = embeddings.shape[0]
maxlen = 10376
batch_size = 150
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 20
embed_size = 512 # for universal sentence encoder
n_class = 12

In [21]:
model = Sequential()

model.add(Dense(max_features, input_shape=(1024,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(max_features, input_shape=(512,)))
#model.add(Dropout(0.1))
model.add(Dense(n_class))
model.add(Activation('sigmoid'))

model.summary()

INFO:plaidml:Opening device "metal_amd_radeon_pro_555x.0"
INFO:plaidml:Opening device "metal_amd_radeon_pro_555x.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10376)             10635400  
_________________________________________________________________
activation_1 (Activation)    (None, 10376)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10376)             0         
_________________________________________________________________
activation_2 (Activation)    (None, 10376)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 10376)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 10376)             107671752 
_________________________________________________________________
dense_3 (Dense)              (None, 12)                124524    
__________

In [22]:
y_train_array = np.asarray(y_train)

In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])


In [25]:
model.fit(embeddings, y_train_array, batch_size=batch_size, epochs=1, validation_split=0.15)

AttributeError: 'Tensor' object has no attribute 'ndim'

In [121]:
import tensorflow_hub as hub
#from cachetools import TTLCache, cached
from tensorflow.keras import layers
import tensorflow as tf

#cache = TTLCache(maxsize=100, ttl=21600)
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
#@cached(cache)

def ELMoEmbedding(x):
    return elmo(tf.reshape(tf.cast(x, tf.string), [-1]), signature="default", as_dict=True)["elmo"]

def test():
  #Test
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())  
    sess.run(tf.tables_initializer())
    embed=sess.run(ELMoEmbedding(tf.constant(['apple juice <pad> <pad> <pad> <pad> <pad> <pad>'])))

    print(embed.shape) # output : 1,8,1024

if __name__=="__main__":
    input_text = layers.Input(shape=(1,), dtype="string", name="Input_Query")
    
    embedding = layers.Lambda(ELMoEmbedding, output_shape=(1024, ), name="Elmo_Embedding")(input_text)
    
    x = layers.Bidirectional(layers.LSTM(1024, return_sequences= False, recurrent_dropout=0.2, dropout=0.2), name="BiLSTM")(embedding)
    x = layers.Dense(500, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(200, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(12, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=[input_text], outputs=outputs, name="tbd")
    model.summary()
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "tbd"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Query (InputLayer)     [(None, 1)]               0         
_________________________________________________________________
Elmo_Embedding (Lambda)      (None, None, 1024)        0         
_________________________________________________________________
BiLSTM (Bidirectional)       (None, 2048)              16785408  
_________________________________________________________________
dense_33 (Dense)             (None, 500)               1024500   
_________________________________________________________________
dropout_22 (Dropout)         (None, 500)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 200)               100200    
_________________________________________________________________
dropout_23 (Dropout)         (None, 200)               0       

In [122]:
model.fit(X_train['preprocessed_comments'], y_train_array, batch_size=batch_size, epochs=1, validation_split=0.15, steps_per_epoch=20)

Train on 20 samples, validate on 1557 samples


FailedPreconditionError: Error while reading resource variable module_13/bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/bias from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/module_13/bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/bias/N10tensorflow3VarE does not exist.
	 [[{{node Elmo_Embedding_11/module_13_apply_default/bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/bias/Read/ReadVariableOp}}]]

In [136]:
session = K.Session()
init = tf.global_variables_initializer()
session.run(init)

AttributeError: module 'keras.backend' has no attribute 'Session'

In [138]:
tf.compat.v1.keras.backend.get_session()

In [139]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [140]:
def ELMoEmbedding(x):
    return elmo(tf.reshape(tf.cast(x, tf.string), [-1]), signature="default", as_dict=True)["elmo"]


In [141]:
input_text = layers.Input(shape=(1,), dtype="string", name="Input_Query")
embedding = layers.Lambda(ELMoEmbedding, output_shape=(1024, ), name="Elmo_Embedding")(input_text)
x = layers.Bidirectional(layers.LSTM(1024, return_sequences= False, recurrent_dropout=0.2, dropout=0.2), name="BiLSTM")(embedding)
x = layers.Dense(8336, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4168, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(12, activation='sigmoid')(x)
model = tf.keras.Model(inputs=[input_text], outputs=outputs, name="tbd")
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "tbd"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Query (InputLayer)     [(None, 1)]               0         
_________________________________________________________________
Elmo_Embedding (Lambda)      (None, None, 1024)        0         
_________________________________________________________________
BiLSTM (Bidirectional)       (None, 2048)              16785408  
_________________________________________________________________
dense_42 (Dense)             (None, 8336)              17080464  
_________________________________________________________________
dropout_28 (Dropout)         (None, 8336)              0         
_________________________________________________________________
dense_43 (Dense)             (None, 4168)              34748616  
_________________________________________________________________
dropout_29 (Dropout)         (None, 4168)              0       

In [142]:
model.fit(X_train['preprocessed_comments'], y_train_array, batch_size=batch_size, epochs=1, validation_split=0.15, steps_per_epoch=20)

Train on 20 samples, validate on 1557 samples


FailedPreconditionError: Error while reading resource variable module_15/bilm/CNN_high_1/b_transform from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/module_15/bilm/CNN_high_1/b_transform/N10tensorflow3VarE does not exist.
	 [[{{node Elmo_Embedding_14/module_15_apply_default/bilm/add_4/ReadVariableOp}}]]